In [71]:
import pandas as pd 
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.stats
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split
%matplotlib inline

In [46]:
train= pd.read_csv("train.csv")
test= pd.read_csv("test.csv")
train.head()

,user_id,user_agent,entry_date,open_date,birthday,id_status,gender,blood_type,pay_method_type,course_name,price_type,entry_from,privilege_name,prefecture,close_flag
0,YKJGBC19356,Mozilla/5.0 (Macintosh; U; PPC Mac OS X 10_9_4...,2016-12-29,2017-01-26,1964-04-10,新規,男性,A,クレジットカード,@nifty光,マンション2年プラン,代理店取次,キャッシュバック還元,鹿児島,0
1,JMFEBB08712,Mozilla/5.0 (compatible; MSIE 9.0; Windows NT ...,2018-04-21,2018-05-29,1980-11-24,新規,女性,O,クレジットカード,@nifty光,ホーム2年プラン,代理店取次,付帯サービス無料,北海道,0
2,EXLFTB26665,Mozilla/5.0 (iPhone; CPU iPhone OS 3_1_3 like ...,2015-11-28,2016-04-05,1972-08-01,既存,女性,AB,クレジットカード,@nifty auひかり,ホーム2年プラン,ノジマ取次,月額割引還元,長野,0
3,MWEEMY19722,Opera/8.10.(X11; Linux i686; nn-NO) Presto/2.9...,2019-07-16,2019-08-15,1966-03-15,新規,男性,B,クレジットカード,@nifty光,マンション2年プラン,代理店取次,キャッシュバック還元,福島,0
4,WWTLYT31226,Mozilla/5.0 (iPad; CPU iPad OS 4_2_1 like Mac ...,2019-09-12,2020-01-06,1988-04-29,新規,男性,A,クレジットカード,@nifty auひかり,マンション3年プラン,WEB代理店,付帯サービス無料,愛知,0


In [47]:
#欠損値の削除
train=train.dropna()
test=test.dropna()

In [48]:
from sklearn.preprocessing import StandardScaler
standard=StandardScaler()

#object型➡datetime64型の変換
train["X"]=pd.to_datetime(train["birthday"],format="%Y-%m-%d")

#年齢のcolumnを追加
train["age"]=2020-train["X"].dt.year

train=train[train["age"]>=20]
train=train[train["age"]<=80]
#標準化
train["age"]=scipy.stats.zscore(train["age"])

#object型➡datetime64型の変換
test["X"]=pd.to_datetime(test["birthday"],format="%Y-%m-%d")

#年齢のcolumnを追加
test["age"]=2020-test["X"].dt.year

test=test[test["age"]>=20]
test=test[test["age"]<=80]
#標準化
test["age"]=scipy.stats.zscore(test["age"])

In [80]:
#インスタンス作成
le=LabelEncoder()
#ラベルの数値化
for column in ["gender","id_status","pay_method_type","course_name","price_type","entry_from","privilege_name"]:
    le=le.fit(train[column])
    train[column]=le.transform(train[column])

for column in ["gender","id_status","pay_method_type","course_name","price_type","entry_from","privilege_name"]:
    le=le.fit(test[column])
    test[column]=le.transform(test[column])

KeyError: 'gender'

In [50]:
train=train.loc[:,["age","pay_method_type","course_name","price_type","privilege_name","close_flag"]]
train.head()

,age,pay_method_type,course_name,price_type,privilege_name,close_flag
0,1.473630,1,2,2,0,0
1,0.133096,1,2,0,1,0
2,0.803363,1,0,0,2,0
3,1.306063,1,2,2,0,0
4,-0.537170,1,0,3,1,0


In [72]:
X=train.loc[:,["age","pay_method_type","course_name","price_type","privilege_name"]].values
y=train.loc[:,["close_flag"]].values


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=1)
#y_trainを1行にする
y_train=np.reshape(y_train,-1)
y_test=np.reshape(y_test,-1)
#モデルの生成
model = LinearSVC(random_state=1)

In [73]:
#モデルの学習
model.fit(X_train,y_train)

LinearSVC(random_state=1)

In [74]:
#yの予測
y_pred=model.predict(X_test)
print(y_test)
print(y_pred)
print(y_test.shape)
print(y_pred.shape)

[0 0 0 ... 0 0 0]
[0 0 0 ... 0 0 0]
(28866,)
(28866,)


In [75]:
from sklearn.metrics import accuracy_score
accuracy_score(y_test, y_pred)

0.9018914986489296

In [79]:
from sklearn.metrics import confusion_matrix
matrix=confusion_matrix(y_test,y_pred)
class_names=["0","1"]
df=pd.DataFrame(matrix,index=class_names,columns=class_names)
df

,0,1
0,26034,0
1,2832,0
